In [1]:
import pandas as pd
import numpy as np

In [2]:
replace_country_key_pair = {'Brunei Darussalam':'Brunei', 'Congo':'Congo (Brazzaville)',
           'Congo DR': 'Congo (Kinshasa)', 'Czech Republic':'Czechia',
           'Korea DPR':'Korea, North', 'Korea Republic of':'Korea, South',
           'Moldova Republic of':'Moldova', 'North Macedonia Republic Of':'North Macedonia','Moldova Republic Of':'Moldova',
           'Russian Federation':'Russia', 'United States of America':'US', 'United States': 'US',
            'Viet Nam':'Vietnam',"CÃ´te d'Ivoire":"Cote d'Ivoire", "kenya": "Kenya", 'South Sudan':'Sudan'}

In [3]:
!dir

 Volume in drive D is non
 Volume Serial Number is A076-5E13

 Directory of D:\Projects\covid19_bigdata_conference_resource\covid19_bigdata_conference_project

06/08/2020  08:30 PM    <DIR>          .
06/08/2020  08:30 PM    <DIR>          ..
06/08/2020  06:21 PM    <DIR>          dataset
06/06/2020  03:18 PM    <DIR>          weather_data
06/07/2020  06:54 PM            22,212 Country_Province_Data_Query_Aun.ipynb
06/08/2020  05:57 PM           421,803 country-wise weather dataset preprocessing.ipynb
06/08/2020  06:50 PM           102,364 preprocessing_attach_weather_to_covid-confirmed-case.ipynb
06/14/2020  03:25 PM            11,651 query_db.ipynb
06/03/2020  02:58 PM             5,633 query_db_test.ipynb
06/08/2020  09:47 PM    <DIR>          .ipynb_checkpoints
06/18/2020  01:44 AM           372,633 preprocessing_all_dataset_to_sql.ipynb
06/11/2020  01:55 PM            19,956 6 conutry should keep.jpg
06/18/2020  01:50 AM            29,201 measures_eda.ipynb
06/14/2020  03:38 PM   

# Connect to DB

In [4]:
import psycopg2
import sys, os
import pandas.io.sql as psql
import sqlalchemy

f = open('db.txt', 'r')
host = f.readline().strip()
dbname = f.readline().strip()
user = f.readline().strip()
password = f.readline().strip()
schema = f.readline().strip()
database = f.readline().strip()
f.close()

# Connect to database (Note: The package psychopg2 is required for Postgres to work with SQLAlchemy)

engine = sqlalchemy.create_engine("postgresql://{}:{}@{}/{}".format(user, password, host, database))
con = engine.connect()

# Verify that there are no existing tables
print(engine.table_names())

['confirmed_weather_no_recov', 'confirmed_no_recov', 'confirmed_weather', 'confirmed', 'measures', 'lockdown_measures', 'us_state_name']


## Covid19-ourworld-dataset

In [84]:
raw_data = pd.read_csv("owid-covid-data.csv")
raw_data.head()

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN


In [87]:
owid_confirmed_df.shape

(21606, 7)

In [85]:
raw_data.describe()

,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
count,2.160600e+04,21606.000000,21606.000000,21606.000000,21221.000000,21221.000000,21221.000000,21221.000000,5.760000e+03,5151.000000,...,19356.000000,19524.000000,19344.000000,12830.000000,19542.000000,20262.000000,15856.000000,15682.000000,8595.000000,17989.000000
mean,2.124004e+04,587.697862,1385.663010,35.250393,579.444586,13.869736,25.164606,0.560496,3.003000e+05,11210.145991,...,9.819503,6.249662,22979.528917,10.374723,245.828003,8.011737,11.270163,32.638551,55.001684,3.217135
std,2.167326e+05,5213.688860,14338.805097,331.496169,1616.993687,63.718910,98.992529,3.451747,1.105118e+06,37640.512689,...,6.458450,4.433764,21292.307966,17.765467,118.877877,4.039355,10.564497,13.228209,31.025621,2.602159
min,0.000000e+00,-2461.000000,0.000000,-1918.000000,0.000000,-265.189000,0.000000,-41.023000,1.000000e+00,1.000000,...,1.144000,0.526000,661.240000,0.100000,79.370000,0.990000,0.100000,7.700000,1.188000,0.100000
25%,7.000000e+00,0.000000,0.000000,0.000000,1.070000,0.000000,0.000000,0.000000,1.005625e+04,606.000000,...,3.941000,2.322000,6426.674000,0.500000,151.089000,5.310000,1.900000,21.400000,24.097000,1.400000
50%,1.160000e+02,2.000000,2.000000,0.000000,40.960000,0.309000,0.315000,0.000000,5.155250e+04,2081.000000,...,7.724000,4.718000,15663.986000,1.600000,233.070000,7.110000,7.000000,31.400000,59.607000,2.600000
75%,1.498750e+03,50.000000,33.000000,1.000000,302.916000,6.369000,5.925000,0.069000,1.859335e+05,6813.500000,...,15.322000,9.829000,35938.374000,10.100000,311.110000,10.080000,19.800000,40.800000,83.841000,4.280000
max,6.348900e+06,127796.000000,380810.000000,10520.000000,20915.566000,4944.376000,1237.551000,200.040000,1.693689e+07,492276.000000,...,27.049000,18.493000,116935.600000,77.600000,724.417000,23.360000,44.000000,78.100000,98.999000,13.800000


In [94]:
raw_data.columns

Index(['iso_code', 'location', 'date', 'total_cases', 'new_cases',
       'total_deaths', 'new_deaths', 'total_cases_per_million',
       'new_cases_per_million', 'total_deaths_per_million',
       'new_deaths_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'tests_units',
       'stringency_index', 'population', 'population_density', 'median_age',
       'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cvd_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'],
      dtype='object')

In [86]:
owid_confirmed_df = raw_data[['iso_code', 'location', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'tests_units', '']]
owid_confirmed_df.isna().sum()

iso_code        64
location         0
date             0
total_cases      0
new_cases        0
total_deaths     0
new_deaths       0
dtype: int64

In [ ]:
owid_confirmed_df

In [93]:
owid_confirmed_df[owid_confirmed_df['location'] == "United States"]

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths
20215,USA,United States,2019-12-31,0,0,0,0
20216,USA,United States,2020-01-01,0,0,0,0
20217,USA,United States,2020-01-02,0,0,0,0
20218,USA,United States,2020-01-03,0,0,0,0
20219,USA,United States,2020-01-04,0,0,0,0
...,...,...,...,...,...,...,...
20366,USA,United States,2020-05-30,1747087,25337,102836,1219
20367,USA,United States,2020-05-31,1770384,23297,103781,945
20368,USA,United States,2020-06-01,1790191,19807,104383,602
20369,USA,United States,2020-06-02,1811277,21086,105147,764


In [95]:
owid_confirmed_df[owid_confirmed_df['location'] == "Thailand"]

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths
19235,THA,Thailand,2019-12-31,0,0,0,0
19236,THA,Thailand,2020-01-01,0,0,0,0
19237,THA,Thailand,2020-01-02,0,0,0,0
19238,THA,Thailand,2020-01-03,0,0,0,0
19239,THA,Thailand,2020-01-04,0,0,0,0
...,...,...,...,...,...,...,...
19379,THA,Thailand,2020-05-30,3076,11,57,0
19380,THA,Thailand,2020-05-31,3081,5,57,0
19381,THA,Thailand,2020-06-01,3081,0,57,0
19382,THA,Thailand,2020-06-02,3082,1,57,0


# US state name

In [5]:
raw_data = pd.read_csv("us_state_names.csv")
raw_data.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [6]:
raw_data.isnull().sum()

State     0
Abbrev    0
Code      0
dtype: int64

In [8]:
us_state_name_df = raw_data.copy()
us_state_name_df.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


# Weather (minTemp, maxTemp, Humidity) dataset

In [5]:
import os

cur_dir = os.getcwd()
weather_data_dir = "weather_data"
weather_data_dir = os.path.join(cur_dir, weather_data_dir)

count = 0
all_country_df = []
for entry in os.scandir(weather_data_dir):
    if(entry.is_file() == True):
        count += 1
        # replace + sign with whitespace
        old_file_path = entry.path
        new_file_path = entry.path.replace("+", " ")
        os.rename(old_file_path, new_file_path)
    
        # read file
        filename = os.path.splitext(entry.name)[0]
        each_country_df = pd.read_csv(new_file_path, index_col=None, header=0)
        each_country_df['country_region'] = filename
        
        all_country_df.append(each_country_df)

all_country_df = pd.concat(all_country_df, ignore_index=True)
all_country_df.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,moonrise,moonset,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,country_region
0,2020-01-01,2,-2,0.9,3.6,1,1,37,11:06 AM,10:41 PM,...,11,84,58,1.7,1025,2,6,23,9,Afghanistan
1,2020-01-02,1,-4,6.6,3.6,1,1,44,11:34 AM,11:36 PM,...,9,100,96,9.0,1027,1,2,21,6,Afghanistan
2,2020-01-03,4,-4,3.2,7.0,1,1,51,12:01 PM,No moonset,...,4,53,75,3.3,1025,4,7,223,3,Afghanistan
3,2020-01-04,4,-3,0.0,8.7,2,2,59,12:29 PM,12:30 AM,...,9,3,41,0.0,1023,4,10,195,6,Afghanistan
4,2020-01-05,-2,-9,2.3,3.6,1,1,66,12:58 PM,01:26 AM,...,6,62,69,3.0,1022,-2,6,129,4,Afghanistan


In [6]:
all_country_df['country_region'] = all_country_df['country_region'].replace(replace_country_key_pair)
all_country_df['country_region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cabo Verde', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica',
       'Croatia', 'Cuba', 'Cyprus', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana',
       'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guyana', 'Haiti',
       'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', '

In [7]:
weather_df = all_country_df[['maxtempC', 'mintempC', 'humidity', 'date_time', 'country_region']]
weather_df.head()

,maxtempC,mintempC,humidity,date_time,country_region
0,2,-2,58,2020-01-01,Afghanistan
1,1,-4,96,2020-01-02,Afghanistan
2,4,-4,75,2020-01-03,Afghanistan
3,4,-3,41,2020-01-04,Afghanistan
4,-2,-9,69,2020-01-05,Afghanistan


# Covid19: Confirmed case with no recovered column

In [8]:
confirmed_no_recov_df = pd.read_csv("dataset/confirmed_no_recov.csv")
confirmed_no_recov_df.head()

,country_region,province_state,date,confirmed,deaths
0,Afghanistan,-,2020-01-22,0,0
1,Albania,-,2020-01-22,0,0
2,Algeria,-,2020-01-22,0,0
3,Andorra,-,2020-01-22,0,0
4,Angola,-,2020-01-22,0,0


In [9]:
confirmed_no_recov_df.isnull().sum()

country_region    0
province_state    0
date              0
confirmed         0
deaths            0
dtype: int64

In [10]:
confirmed_no_recov_df['country_region'] = confirmed_no_recov_df['country_region'].replace(replace_country_key_pair)
confirmed_no_recov_df['country_region'].nunique()

187

In [11]:
merged_weather_covid_df = pd.merge(confirmed_no_recov_df, weather_df, how='left', left_on=['country_region', 'date'], right_on=['country_region', 'date_time'])
merged_weather_covid_df

,country_region,province_state,date,confirmed,deaths,maxtempC,mintempC,humidity,date_time
0,Afghanistan,-,2020-01-22,0,0,-1.0,-12.0,65.0,2020-01-22
1,Albania,-,2020-01-22,0,0,11.0,6.0,51.0,2020-01-22
2,Algeria,-,2020-01-22,0,0,19.0,13.0,50.0,2020-01-22
3,Andorra,-,2020-01-22,0,0,0.0,-8.0,98.0,2020-01-22
4,Angola,-,2020-01-22,0,0,30.0,26.0,78.0,2020-01-22
...,...,...,...,...,...,...,...,...,...
43603,West Bank and Gaza,-,2020-06-07,472,3,NaN,NaN,NaN,NaN
43604,Western Sahara,-,2020-06-07,9,1,27.0,19.0,69.0,2020-06-07
43605,Yemen,-,2020-06-07,484,112,26.0,19.0,28.0,2020-06-07
43606,Zambia,-,2020-06-07,1089,7,22.0,13.0,63.0,2020-06-07


In [12]:
merged_weather_covid_df[merged_weather_covid_df['maxtempC'].isnull()].groupby('country_region').count()

,province_state,date,confirmed,deaths,maxtempC,mintempC,humidity,date_time
country_region,,,,,,,,
Diamond Princess,138,138,138,138,0,0,0,0
Holy See,138,138,138,138,0,0,0,0
MS Zaandam,138,138,138,138,0,0,0,0
Taiwan*,138,138,138,138,0,0,0,0
Timor-Leste,138,138,138,138,0,0,0,0
West Bank and Gaza,138,138,138,138,0,0,0,0


In [13]:
merged_weather_covid_df = merged_weather_covid_df.drop(['date_time'], axis=1)
merged_weather_covid_df

,country_region,province_state,date,confirmed,deaths,maxtempC,mintempC,humidity
0,Afghanistan,-,2020-01-22,0,0,-1.0,-12.0,65.0
1,Albania,-,2020-01-22,0,0,11.0,6.0,51.0
2,Algeria,-,2020-01-22,0,0,19.0,13.0,50.0
3,Andorra,-,2020-01-22,0,0,0.0,-8.0,98.0
4,Angola,-,2020-01-22,0,0,30.0,26.0,78.0
...,...,...,...,...,...,...,...,...
43603,West Bank and Gaza,-,2020-06-07,472,3,NaN,NaN,NaN
43604,Western Sahara,-,2020-06-07,9,1,27.0,19.0,69.0
43605,Yemen,-,2020-06-07,484,112,26.0,19.0,28.0
43606,Zambia,-,2020-06-07,1089,7,22.0,13.0,63.0


In [14]:
merged_weather_covid_df = merged_weather_covid_df.fillna("*")
merged_weather_covid_df.isnull().sum()

country_region    0
province_state    0
date              0
confirmed         0
deaths            0
maxtempC          0
mintempC          0
humidity          0
dtype: int64

In [15]:
merged_weather_covid_df

,country_region,province_state,date,confirmed,deaths,maxtempC,mintempC,humidity
0,Afghanistan,-,2020-01-22,0,0,-1,-12,65
1,Albania,-,2020-01-22,0,0,11,6,51
2,Algeria,-,2020-01-22,0,0,19,13,50
3,Andorra,-,2020-01-22,0,0,0,-8,98
4,Angola,-,2020-01-22,0,0,30,26,78
...,...,...,...,...,...,...,...,...
43603,West Bank and Gaza,-,2020-06-07,472,3,-,-,-
43604,Western Sahara,-,2020-06-07,9,1,27,19,69
43605,Yemen,-,2020-06-07,484,112,26,19,28
43606,Zambia,-,2020-06-07,1089,7,22,13,63


In [20]:
merged_weather_covid_df[merged_weather_covid_df['country_region'] == 'Colombia'].tail(10)

,country_region,province_state,date,confirmed,deaths,maxtempC,mintempC,humidity
40537,Colombia,-,2020-05-29,25406,855,20,10,84
40853,Colombia,-,2020-05-30,26734,891,19,10,85
41169,Colombia,-,2020-05-31,27219,916,20,11,86
41485,Colombia,-,2020-06-01,29384,963,19,11,85
41801,Colombia,-,2020-06-02,30593,1014,21,11,80
42117,Colombia,-,2020-06-03,31935,1057,20,11,85
42433,Colombia,-,2020-06-04,33466,1099,20,10,82
42749,Colombia,-,2020-06-05,36759,1204,19,10,84
43065,Colombia,-,2020-06-06,36759,1204,20,12,82
43381,Colombia,-,2020-06-07,38149,1265,19,11,78


In [17]:
table_name = 'confirmed_weather_no_recov'
# use "multi" method for insert many value in one statement sql
merged_weather_covid_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

In [18]:
table_name = 'confirmed_no_recov'
# use "multi" method for insert many value in one statement sql
confirmed_no_recov_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

# covid-title-dataset - confirmed_case

In [21]:
confirmed_df = pd.read_csv("dataset/confirmed.csv")
confirmed_df.head()

,country_region,date,confirmed,death,recovered
0,Afghanistan,2020-01-22,0,0,0
1,Afghanistan,2020-01-23,0,0,0
2,Afghanistan,2020-01-24,0,0,0
3,Afghanistan,2020-01-25,0,0,0
4,Afghanistan,2020-01-26,0,0,0


In [22]:
confirmed_df.isnull().sum()

country_region    0
date              0
confirmed         0
death             0
recovered         0
dtype: int64

In [23]:
confirmed_df['country_region'] = confirmed_df['country_region'].replace(replace_country_key_pair)

In [24]:
confirmed_df['country_region'].nunique()

187

In [25]:
merged_confirmed_weather_df = pd.merge(confirmed_df, weather_df, how='left', left_on=['country_region', 'date'], right_on=['country_region', 'date_time'])
merged_confirmed_weather_df

,country_region,date,confirmed,death,recovered,maxtempC,mintempC,humidity,date_time
0,Afghanistan,2020-01-22,0,0,0,-1.0,-12.0,65.0,2020-01-22
1,Afghanistan,2020-01-23,0,0,0,-3.0,-13.0,59.0,2020-01-23
2,Afghanistan,2020-01-24,0,0,0,0.0,-9.0,71.0,2020-01-24
3,Afghanistan,2020-01-25,0,0,0,0.0,-8.0,79.0,2020-01-25
4,Afghanistan,2020-01-26,0,0,0,-1.0,-10.0,64.0,2020-01-26
...,...,...,...,...,...,...,...,...,...
25939,Zimbabwe,2020-06-03,222,4,29,23.0,10.0,56.0,2020-06-03
25940,Zimbabwe,2020-06-04,237,4,31,23.0,10.0,63.0,2020-06-04
25941,Zimbabwe,2020-06-05,265,4,33,25.0,10.0,58.0,2020-06-05
25942,Zimbabwe,2020-06-06,279,4,33,21.0,10.0,64.0,2020-06-06


In [26]:
merged_confirmed_weather_df[merged_confirmed_weather_df['maxtempC'].isnull()].groupby('country_region').count()

,date,confirmed,death,recovered,maxtempC,mintempC,humidity,date_time
country_region,,,,,,,,
Diamond Princess,138,138,138,138,0,0,0,0
Holy See,138,138,138,138,0,0,0,0
MS Zaandam,138,138,138,138,0,0,0,0
Taiwan*,138,138,138,138,0,0,0,0
Timor-Leste,138,138,138,138,0,0,0,0
West Bank and Gaza,138,138,138,138,0,0,0,0


In [27]:
merged_confirmed_weather_df = merged_confirmed_weather_df.drop(['date_time'], axis=1)

In [28]:
merged_confirmed_weather_df = merged_confirmed_weather_df.fillna("*")
merged_confirmed_weather_df

,country_region,date,confirmed,death,recovered,maxtempC,mintempC,humidity
0,Afghanistan,2020-01-22,0,0,0,-1,-12,65
1,Afghanistan,2020-01-23,0,0,0,-3,-13,59
2,Afghanistan,2020-01-24,0,0,0,0,-9,71
3,Afghanistan,2020-01-25,0,0,0,0,-8,79
4,Afghanistan,2020-01-26,0,0,0,-1,-10,64
...,...,...,...,...,...,...,...,...
25939,Zimbabwe,2020-06-03,222,4,29,23,10,56
25940,Zimbabwe,2020-06-04,237,4,31,23,10,63
25941,Zimbabwe,2020-06-05,265,4,33,25,10,58
25942,Zimbabwe,2020-06-06,279,4,33,21,10,64


In [29]:
table_name = 'confirmed_weather'
# use "multi" method for insert many value in one statement sql
merged_confirmed_weather_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

In [30]:
table_name = 'confirmed'
# use "multi" method for insert many value in one statement sql
confirmed_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

# Johns Hopkins Covid19 Dataset

In [74]:
raw_data = pd.read_csv("dataset/covid19-jhc-dataset.csv")
raw_data.head()

,province_state,country_region,date,latitude,longitude,location_geom,confirmed,deaths,recovered,active,fips,admin2,combined_key
0,NaN,UK,2020-02-05,NaN,NaN,NaN,2.0,0.0,0.0,NaN,NaN,NaN,NaN
1,NaN,UK,2020-02-14,NaN,NaN,NaN,9.0,0.0,1.0,NaN,NaN,NaN,NaN
2,NaN,UK,2020-02-23,NaN,NaN,NaN,9.0,0.0,8.0,NaN,NaN,NaN,NaN
3,NaN,UK,2020-02-29,NaN,NaN,NaN,23.0,0.0,8.0,NaN,NaN,NaN,NaN
4,NaN,UK,2020-02-04,NaN,NaN,NaN,2.0,0.0,0.0,NaN,NaN,NaN,NaN


In [75]:
raw_data.describe()

,latitude,longitude,confirmed,deaths,recovered,active,fips
count,233487.000000,233487.000000,240293.000000,239871.000000,239924.000000,232695.000000,209564.000000
mean,36.397411,-78.917916,963.081351,62.259227,329.870013,560.429665,31153.778931
std,10.865079,44.128993,8665.802649,845.736961,5410.746670,6855.307824,16934.041171
min,-52.368000,-170.132000,0.000000,0.000000,0.000000,-412240.000000,66.000000
25%,33.720769,-96.223192,4.000000,0.000000,0.000000,0.000000,18135.000000
50%,37.877361,-87.728840,19.000000,0.000000,0.000000,11.000000,29151.000000
75%,41.608600,-80.640165,115.000000,3.000000,0.000000,78.000000,46039.000000
max,72.000000,178.065000,405843.000000,39369.000000,463868.000000,238616.000000,99999.000000


In [76]:
confirmed_df = raw_data[['province_state', 'country_region', 'date', 'confirmed', 'deaths', 'recovered', 'active']]
confirmed_df.isna().sum()

province_state    16301
country_region        0
date                  0
confirmed            19
deaths              441
recovered           388
active             7617
dtype: int64

In [77]:
values = {'province_state': "-", 'confirmed': 0, 'deaths': 0, 'recovered': 0, 'active': 0}
confirmed_df = confirmed_df.fillna(value=values)
confirmed_df

,province_state,country_region,date,confirmed,deaths,recovered,active
0,-,UK,2020-02-05,2.0,0.0,0.0,0.0
1,-,UK,2020-02-14,9.0,0.0,1.0,0.0
2,-,UK,2020-02-23,9.0,0.0,8.0,0.0
3,-,UK,2020-02-29,23.0,0.0,8.0,0.0
4,-,UK,2020-02-04,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
240307,-,Saint Vincent and the Grenadines,2020-03-17,1.0,0.0,0.0,0.0
240308,-,Saint Vincent and the Grenadines,2020-03-16,1.0,0.0,0.0,0.0
240309,-,Saint Vincent and the Grenadines,2020-03-15,1.0,0.0,0.0,0.0
240310,-,Saint Vincent and the Grenadines,2020-03-18,1.0,0.0,0.0,0.0


## Policies dataset

1. filter only date-implement
2. full lockdown and partial lockdown to lockdown
3. groupby COUNTRY, ADMIN_LEVEL_NAME, CATEGORY, MEASURE, DATE_IMPLEMENTED, 

### Import dataset (2 dataset)

In [76]:
raw_measure_data = pd.read_excel('dataset/acaps_covid19_government_measures_datasets.xlsx', sheet_name='Database')
raw_measure_data = raw_measure_data[['COUNTRY', 'REGION', 'LOG_TYPE', 'CATEGORY', 'MEASURE', 'ADMIN_LEVEL_NAME', 'DATE_IMPLEMENTED', 'ENTRY_DATE']]
raw_measure_data

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
0,Afghanistan,Asia,Introduction / extension of measures,Public health measures,Health screenings in airports and border cross...,NaN,2020-02-12,2020-03-14
1,Afghanistan,Asia,Introduction / extension of measures,Public health measures,Isolation and quarantine policies,Kabul,2020-02-12,2020-03-14
2,Afghanistan,Asia,Introduction / extension of measures,Public health measures,Awareness campaigns,NaN,2020-02-12,2020-03-14
3,Afghanistan,Asia,Introduction / extension of measures,Governance and socio-economic measures,Emergency administrative structures activated ...,NaN,2020-02-12,2020-03-14
4,Afghanistan,Asia,Introduction / extension of measures,Social distancing,Limit public gatherings,NaN,2020-03-12,2020-03-14
...,...,...,...,...,...,...,...,...
12533,Zimbabwe,Africa,Introduction / extension of measures,Lockdown,Partial lockdown,NaN,2020-05-17,2020-05-25
12534,Zimbabwe,Africa,Introduction / extension of measures,Public health measures,Requirement to wear protective gear in public,NaN,2020-05-04,2020-05-25
12535,Zimbabwe,Africa,Introduction / extension of measures,Lockdown,Partial lockdown,NaN,2020-05-04,2020-05-25
12536,Zimbabwe,Africa,Phase-out measure,Social distancing,Limit public gatherings,NaN,2020-05-17,2020-05-25


In [77]:
raw_kaggle_lockdowndate_df = pd.read_csv('dataset/new_lockdown_country.csv')
raw_kaggle_lockdowndate_df

,COUNTRY,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,MEASURE,REGION,LOG_TYPE,CATEGORY,ENTRY_DATE
0,Costa Rica,NaN,3/23/2020,lockdown,NaN,Introduction / extension of measures,Lockdown,3/23/2020
1,Denmark,NaN,3/11/2020,lockdown,NaN,Introduction / extension of measures,Lockdown,3/11/2020
2,Netherlands,NaN,3/16/2020,lockdown,NaN,Introduction / extension of measures,Lockdown,3/16/2020
3,Norway,NaN,3/12/2020,lockdown,NaN,Introduction / extension of measures,Lockdown,3/12/2020
4,Pakistan,Gilgit-Baltistan,3/22/2020,lockdown,Gilgit-Baltistan,Introduction / extension of measures,Lockdown,3/22/2020
5,Saudi Arabia,NaN,3/9/2020,lockdown,NaN,Introduction / extension of measures,Lockdown,3/9/2020
6,United States,California,3/19/2020,lockdown,California,Introduction / extension of measures,Lockdown,3/19/2020
7,United States,"Clark County, NV",3/20/2020,lockdown,"Clark County, NV",Introduction / extension of measures,Lockdown,3/20/2020
8,United States,Connecticut,3/23/2020,lockdown,Connecticut,Introduction / extension of measures,Lockdown,3/23/2020
9,United States,Illinois,3/21/2020,lockdown,Illinois,Introduction / extension of measures,Lockdown,3/21/2020


### Preprocessing - date format

In [78]:
preprocessing_mesures_df = raw_measure_data[pd.notna(raw_measure_data['DATE_IMPLEMENTED'])]
preprocessing_mesures_df.head(5)

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
0,Afghanistan,Asia,Introduction / extension of measures,Public health measures,Health screenings in airports and border cross...,NaN,2020-02-12,2020-03-14
1,Afghanistan,Asia,Introduction / extension of measures,Public health measures,Isolation and quarantine policies,Kabul,2020-02-12,2020-03-14
2,Afghanistan,Asia,Introduction / extension of measures,Public health measures,Awareness campaigns,NaN,2020-02-12,2020-03-14
3,Afghanistan,Asia,Introduction / extension of measures,Governance and socio-economic measures,Emergency administrative structures activated ...,NaN,2020-02-12,2020-03-14
4,Afghanistan,Asia,Introduction / extension of measures,Social distancing,Limit public gatherings,NaN,2020-03-12,2020-03-14


In [79]:
preprocessing_mesures_df['MEASURE'] = preprocessing_mesures_df['MEASURE'].str.lower()
preprocessing_mesures_df['MEASURE'] = preprocessing_mesures_df['MEASURE'].replace({'partial lockdown': 'lockdown', 'full lockdown': 'lockdown'})
preprocessing_mesures_df

C:\Users\Narawit Tubtimtoe\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Narawit Tubtimtoe\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
0,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,NaN,2020-02-12,2020-03-14
1,Afghanistan,Asia,Introduction / extension of measures,Public health measures,isolation and quarantine policies,Kabul,2020-02-12,2020-03-14
2,Afghanistan,Asia,Introduction / extension of measures,Public health measures,awareness campaigns,NaN,2020-02-12,2020-03-14
3,Afghanistan,Asia,Introduction / extension of measures,Governance and socio-economic measures,emergency administrative structures activated ...,NaN,2020-02-12,2020-03-14
4,Afghanistan,Asia,Introduction / extension of measures,Social distancing,limit public gatherings,NaN,2020-03-12,2020-03-14
...,...,...,...,...,...,...,...,...
12533,Zimbabwe,Africa,Introduction / extension of measures,Lockdown,lockdown,NaN,2020-05-17,2020-05-25
12534,Zimbabwe,Africa,Introduction / extension of measures,Public health measures,requirement to wear protective gear in public,NaN,2020-05-04,2020-05-25
12535,Zimbabwe,Africa,Introduction / extension of measures,Lockdown,lockdown,NaN,2020-05-04,2020-05-25
12536,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,NaN,2020-05-17,2020-05-25


In [80]:
raw_kaggle_lockdowndate_df['DATE_IMPLEMENTED'] = pd.to_datetime(raw_kaggle_lockdowndate_df['DATE_IMPLEMENTED'])
raw_kaggle_lockdowndate_df['DATE_IMPLEMENTED'].dt.strftime('%Y-%m-%d')
raw_kaggle_lockdowndate_df['DATE_IMPLEMENTED'].head(10)

0   2020-03-23
1   2020-03-11
2   2020-03-16
3   2020-03-12
4   2020-03-22
5   2020-03-09
6   2020-03-19
7   2020-03-20
8   2020-03-23
9   2020-03-21
Name: DATE_IMPLEMENTED, dtype: datetime64[ns]

### Concat both dataset into one dataset

In [81]:
concated_measure_df = pd.concat([preprocessing_mesures_df, raw_kaggle_lockdowndate_df])
concated_measure_df = concated_measure_df.sort_values(by=['COUNTRY', 'DATE_IMPLEMENTED'])
concated_measure_df.shape

(12309, 8)

### Modify country name in concated DF to have the same country name

In [82]:
concated_measure_df['COUNTRY'] = concated_measure_df['COUNTRY'].replace(replace_country_key_pair)

print('number of unique country: {}'.format(concated_measure_df['COUNTRY'].nunique()))
print(concated_measure_df['COUNTRY'].unique())

number of unique country: 192
['Afghanistan' 'Albania' 'Algeria' 'Angola' 'Antigua and Barbuda'
 'Argentina' 'Armenia' 'Australia' 'Austria' 'Azerbaijan' 'Bahamas'
 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin'
 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Brunei'
 'Bulgaria' 'Burkina Faso' 'Burundi' 'Cabo Verde' 'Cambodia' 'Cameroon'
 'Canada' 'Central African Republic' 'Chad' 'Chile' 'China'
 'China, Hong Kong Special Administrative Region' 'Colombia' 'Comoros'
 'Congo (Brazzaville)' 'Congo (Kinshasa)' 'Costa Rica' 'Croatia' 'Cuba'
 'Cyprus' 'Czechia' "Côte d'Ivoire" 'Denmark' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Honduras'
 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq' 'Ireland' 'Israel'

### Preprocessing

####  ADMIN_LEVEL_NAME is "-" when NaN & other NaN columns is *

In [83]:
concated_measure_df['ADMIN_LEVEL_NAME'] = concated_measure_df['ADMIN_LEVEL_NAME'].fillna("-")
measures_df = concated_measure_df.fillna("*")
measures_df

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
11,Afghanistan,Asia,Introduction / extension of measures,Public health measures,awareness campaigns,-,2020-01-24,2020-04-07 00:00:00
12,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-26,2020-04-07 00:00:00
13,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,international flights suspension,-,2020-01-27,2020-04-07 00:00:00
14,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-27,2020-04-07 00:00:00
10,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,border checks,-,2020-02-01,2020-03-14 00:00:00
...,...,...,...,...,...,...,...,...
12531,Zimbabwe,Africa,Phase-out measure,Social distancing,closure of businesses and public services,-,2020-05-16,2020-05-25 00:00:00
12533,Zimbabwe,Africa,Introduction / extension of measures,Lockdown,lockdown,-,2020-05-17,2020-05-25 00:00:00
12536,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,-,2020-05-17,2020-05-25 00:00:00
12537,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,-,2020-05-17,2020-05-25 00:00:00


####  handle '\xa0' in measure name

In [84]:
measures_df['MEASURE'] = measures_df['MEASURE'].str.replace('\xa0', '')
measures_df['MEASURE'].unique()

array(['awareness campaigns',
       'health screenings in airports and border crossings',
       'international flights suspension', 'border checks',
       'strengthening the public health system',
       'isolation and quarantine policies',
       'emergency administrative structures activated or established',
       'surveillance and monitoring',
       'other public health measures enforced', 'border closure',
       'domestic travel restrictions', 'state of emergency declared',
       'general recommendations', 'schools closure',
       'limit public gatherings', 'limit product imports/exports',
       'lockdown', 'changes in prison-related policies',
       'checkpoints within the country', 'economic measures',
       'closure of businesses and public services', 'military deployment',
       'curfews', 'requirement to wear protective gear in public',
       'visa restrictions',
       'lockdown of refugee/idp camps or other minorities',
       'additional health/documents requir

### Split data into two group (keep province and not keep province)

In [85]:
keep_provincestate_list = ['Canada', 'Australia', 'China', 'France', 'US', 'United Kingdom', 'United States']

In [86]:
measures_df.loc[measures_df['COUNTRY'] == 'China, Hong Kong Special Administrative Region', 'ADMIN_LEVEL_NAME'] = 'Hong Kong'
measures_df['COUNTRY'] = measures_df['COUNTRY'].replace({'China, Hong Kong Special Administrative Region': 'China'})

In [87]:
keepprovince_measures_df = measures_df[measures_df['COUNTRY'].isin(keep_provincestate_list)]
keepprovince_measures_df

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
476,Australia,Pacific,Introduction / extension of measures,Governance and socio-economic measures,emergency administrative structures activated ...,-,2020-02-17,2020-03-14 00:00:00
471,Australia,Pacific,Introduction / extension of measures,Movement restrictions,visa restrictions,-,2020-03-01,2020-03-14 00:00:00
472,Australia,Pacific,Introduction / extension of measures,Public health measures,isolation and quarantine policies,-,2020-03-01,2020-03-14 00:00:00
474,Australia,Pacific,Introduction / extension of measures,Governance and socio-economic measures,economic measures,-,2020-03-01,2020-03-14 00:00:00
475,Australia,Pacific,Introduction / extension of measures,Public health measures,strengthening the public health system,-,2020-03-12,2020-03-14 00:00:00
...,...,...,...,...,...,...,...,...
12176,US,Americas,Introduction / extension of measures,Public health measures,strengthening the public health system,-,2020-04-20,2020-04-21 00:00:00
12177,US,Americas,Introduction / extension of measures,Public health measures,strengthening the public health system,-,2020-04-20,2020-04-21 00:00:00
12100,US,Americas,Introduction / extension of measures,Movement restrictions,visa restrictions,-,2020-04-22,2020-04-21 00:00:00
12196,US,Americas,Introduction / extension of measures,Movement restrictions,surveillance and monitoring,-,2020-04-29,2020-04-21 00:00:00


In [88]:
notkeepprovince_measures_df = measures_df[~measures_df['COUNTRY'].isin(keep_provincestate_list)]
notkeepprovince_measures_df

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
11,Afghanistan,Asia,Introduction / extension of measures,Public health measures,awareness campaigns,-,2020-01-24,2020-04-07 00:00:00
12,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-26,2020-04-07 00:00:00
13,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,international flights suspension,-,2020-01-27,2020-04-07 00:00:00
14,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-27,2020-04-07 00:00:00
10,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,border checks,-,2020-02-01,2020-03-14 00:00:00
...,...,...,...,...,...,...,...,...
12531,Zimbabwe,Africa,Phase-out measure,Social distancing,closure of businesses and public services,-,2020-05-16,2020-05-25 00:00:00
12533,Zimbabwe,Africa,Introduction / extension of measures,Lockdown,lockdown,-,2020-05-17,2020-05-25 00:00:00
12536,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,-,2020-05-17,2020-05-25 00:00:00
12537,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,-,2020-05-17,2020-05-25 00:00:00


### Preprocessing - notkeepprovince group

logic: มาตรการไหนใช้ในประเทศนั้นเยอะๆ คือเกิน 5 เรา assume ว่ามีหลายจังหวัดที่ใช้มาตรการนั้น เหมือนกับว่าทั้งประเทศใช้มาตรการนั้นเหมือนกัน โดยใช้ลอจิกนี้เฉพาะที่ admin_level_name != "-"

In [89]:
country_notkeepprovince_measures_df = notkeepprovince_measures_df[notkeepprovince_measures_df['ADMIN_LEVEL_NAME'] == "-"]
country_notkeepprovince_measures_df

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
11,Afghanistan,Asia,Introduction / extension of measures,Public health measures,awareness campaigns,-,2020-01-24,2020-04-07 00:00:00
12,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-26,2020-04-07 00:00:00
13,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,international flights suspension,-,2020-01-27,2020-04-07 00:00:00
14,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-27,2020-04-07 00:00:00
10,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,border checks,-,2020-02-01,2020-03-14 00:00:00
...,...,...,...,...,...,...,...,...
12531,Zimbabwe,Africa,Phase-out measure,Social distancing,closure of businesses and public services,-,2020-05-16,2020-05-25 00:00:00
12533,Zimbabwe,Africa,Introduction / extension of measures,Lockdown,lockdown,-,2020-05-17,2020-05-25 00:00:00
12536,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,-,2020-05-17,2020-05-25 00:00:00
12537,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,-,2020-05-17,2020-05-25 00:00:00


In [90]:
province_notkeepprovince_measures_df = notkeepprovince_measures_df[notkeepprovince_measures_df['ADMIN_LEVEL_NAME'] != "-"]
province_notkeepprovince_measures_df

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
15,Afghanistan,Asia,Introduction / extension of measures,Public health measures,strengthening the public health system,Kabul,2020-02-02,2020-04-07 00:00:00
1,Afghanistan,Asia,Introduction / extension of measures,Public health measures,isolation and quarantine policies,Kabul,2020-02-12,2020-03-14 00:00:00
21,Afghanistan,Asia,Introduction / extension of measures,Public health measures,isolation and quarantine policies,Kabul,2020-02-12,2020-04-07 00:00:00
26,Afghanistan,Asia,Introduction / extension of measures,Governance and socio-economic measures,state of emergency declared,Herat Province,2020-02-24,2020-04-07 00:00:00
6,Afghanistan,Asia,Introduction / extension of measures,Social distancing,schools closure,Herat,2020-03-01,2020-03-14 00:00:00
...,...,...,...,...,...,...,...,...
12470,Yemen,Middle East,Introduction / extension of measures,Governance and socio-economic measures,emergency administrative structures activated ...,Aden,2020-05-11,2020-05-12 00:00:00
12472,Yemen,Middle East,Introduction / extension of measures,Movement restrictions,domestic travel restrictions,Aden,2020-05-11,2020-05-22 00:00:00
12474,Yemen,Middle East,Introduction / extension of measures,Movement restrictions,domestic travel restrictions,Marib,2020-05-24,2020-05-29 00:00:00
12475,Yemen,Middle East,Introduction / extension of measures,Movement restrictions,curfews,Marib,2020-05-24,2020-05-29 00:00:00


#### admin_level_name > province

In [91]:
count_province_notkeepprovince_measures_series = province_notkeepprovince_measures_df.groupby(['COUNTRY', 'MEASURE'])['MEASURE'].count()
count_province_notkeepprovince_measures_series.head(20)

COUNTRY      MEASURE                                           
Afghanistan  border checks                                         1
             health screenings in airports and border crossings    1
             isolation and quarantine policies                     2
             limit public gatherings                               1
             lockdown                                              1
             schools closure                                       1
             state of emergency declared                           1
             strengthening the public health system                5
Albania      domestic travel restrictions                          1
Algeria      checkpoints within the country                        1
             curfews                                               1
             international flights suspension                      1
             lockdown                                              4
             requirement to wear protec

In [92]:
grouped_province_notkeepprovince_measures_df = province_notkeepprovince_measures_df.groupby(['COUNTRY', 'MEASURE', 'LOG_TYPE']).first()
grouped_province_notkeepprovince_measures_df

REGION  \
COUNTRY     MEASURE                                            LOG_TYPE                                            
Afghanistan border checks                                      Introduction / extension of measures         Asia   
            health screenings in airports and border crossings Introduction / extension of measures         Asia   
            isolation and quarantine policies                  Introduction / extension of measures         Asia   
            limit public gatherings                            Introduction / extension of measures         Asia   
            lockdown                                           Introduction / extension of measures         Asia   
...                                                                                                          ...   
Yemen       lockdown                                           Introduction / extension of measures  Middle East   
            schools closure                                    Introduction / extension of measures  Middle East   
            strengthening the public health system             Introduction / extension of measures  Middle East   
            surveillance and monitoring                        Introduction / extension of measures  Middle East   
Zambia      domestic travel restrictions                       Introduction / extension of measures       Africa   

                                                                                                                   CATEGORY  \
COUNTRY     MEASURE                                            LOG_TYPE                                                       
Afghanistan border checks                                      Introduction / extension of measures   Movement restrictions   
            health screenings in airports and border crossings Introduction / extension of measures  Public health measures   
            isolation and quarantine policies                  Introduction / extension of measures  Public health measures   
            limit public gatherings                            Introduction / extension of measures       Social distancing   
            lockdown                                           Introduction / extension of measures                Lockdown   
...                                                                                                                     ...   
Yemen       lockdown                                           Introduction / extension of measures                Lockdown   
            schools closure                                    Introduction / extension of measures       Social distancing   
            strengthening the public health system             Introduction / extension of measures  Public health measures   
            surveillance and monitoring                        Introduction / extension of measures   Movement restrictions   
Zambia      domestic travel restrictions                       Introduction / extension of measures   Movement restrictions   

                                                                                                                          ADMIN_LEVEL_NAME  \
COUNTRY     MEASURE                                            LOG_TYPE                                                                      
Afghanistan border checks                                      Introduction / extension of measures                        Midday Province   
            health screenings in airports and border crossings Introduction / extension of measures                         Herat Province   
            isolation and quarantine policies                  Introduction / extension of measures                                  Kabul   
            limit public gatherings                            Introduction / extension of measures                                  Herat   
            lockdown                                           Introduction / extension of me

In [93]:
grouped_province_notkeepprovince_measures_df = grouped_province_notkeepprovince_measures_df.merge(count_province_notkeepprovince_measures_series, left_index=True, right_index=True)
grouped_province_notkeepprovince_measures_df = grouped_province_notkeepprovince_measures_df.rename(columns={'MEASURE': 'MEASURE_COUNT'})
grouped_province_notkeepprovince_measures_df = grouped_province_notkeepprovince_measures_df.reset_index()
filtered_province_notkeepprovince_measures_df = grouped_province_notkeepprovince_measures_df[grouped_province_notkeepprovince_measures_df['MEASURE_COUNT'] > 4]
print('Shape: {}'.format(filtered_province_notkeepprovince_measures_df.shape))
filtered_province_notkeepprovince_measures_df.head()

Shape: (77, 9)


,COUNTRY,MEASURE,LOG_TYPE,REGION,CATEGORY,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE,MEASURE_COUNT
7,Afghanistan,strengthening the public health system,Introduction / extension of measures,Asia,Public health measures,Kabul,2020-02-02,2020-04-07 00:00:00,5
24,Angola,strengthening the public health system,Introduction / extension of measures,Africa,Public health measures,Lubango,2020-03-26,2020-03-27 00:00:00,13
37,Austria,closure of businesses and public services,Introduction / extension of measures,Europe,Social distancing,Salzburg,2020-04-10,2020-04-25 00:00:00,8
38,Austria,closure of businesses and public services,Phase-out measure,Europe,Social distancing,Upper Austria,2020-04-03,2020-04-25 00:00:00,8
39,Austria,domestic travel restrictions,Introduction / extension of measures,Europe,Movement restrictions,Tyrol,2020-03-13,2020-03-15 00:00:00,6


In [94]:
filtered_province_notkeepprovince_measures_df = filtered_province_notkeepprovince_measures_df.drop(['MEASURE_COUNT'], axis=1)
filtered_province_notkeepprovince_measures_df

,COUNTRY,MEASURE,LOG_TYPE,REGION,CATEGORY,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
7,Afghanistan,strengthening the public health system,Introduction / extension of measures,Asia,Public health measures,Kabul,2020-02-02,2020-04-07 00:00:00
24,Angola,strengthening the public health system,Introduction / extension of measures,Africa,Public health measures,Lubango,2020-03-26,2020-03-27 00:00:00
37,Austria,closure of businesses and public services,Introduction / extension of measures,Europe,Social distancing,Salzburg,2020-04-10,2020-04-25 00:00:00
38,Austria,closure of businesses and public services,Phase-out measure,Europe,Social distancing,Upper Austria,2020-04-03,2020-04-25 00:00:00
39,Austria,domestic travel restrictions,Introduction / extension of measures,Europe,Movement restrictions,Tyrol,2020-03-13,2020-03-15 00:00:00
...,...,...,...,...,...,...,...,...
492,Switzerland,closure of businesses and public services,Introduction / extension of measures,Europe,Social distancing,Jura,2020-04-24,2020-04-25 00:00:00
493,Switzerland,closure of businesses and public services,Phase-out measure,Europe,Social distancing,Wallis,2020-04-20,2020-04-25 00:00:00
494,Switzerland,economic measures,Introduction / extension of measures,Europe,Governance and socio-economic measures,Thurgau,2020-03-13,2020-04-25 00:00:00
554,Yemen,closure of businesses and public services,Introduction / extension of measures,Middle East,Social distancing,Internationally Recognized Government,2020-03-16,2020-04-09 00:00:00


#### admin_level_name > country

In [95]:
country_notkeepprovince_measures_df

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
11,Afghanistan,Asia,Introduction / extension of measures,Public health measures,awareness campaigns,-,2020-01-24,2020-04-07 00:00:00
12,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-26,2020-04-07 00:00:00
13,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,international flights suspension,-,2020-01-27,2020-04-07 00:00:00
14,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-27,2020-04-07 00:00:00
10,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,border checks,-,2020-02-01,2020-03-14 00:00:00
...,...,...,...,...,...,...,...,...
12531,Zimbabwe,Africa,Phase-out measure,Social distancing,closure of businesses and public services,-,2020-05-16,2020-05-25 00:00:00
12533,Zimbabwe,Africa,Introduction / extension of measures,Lockdown,lockdown,-,2020-05-17,2020-05-25 00:00:00
12536,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,-,2020-05-17,2020-05-25 00:00:00
12537,Zimbabwe,Africa,Phase-out measure,Social distancing,limit public gatherings,-,2020-05-17,2020-05-25 00:00:00


In [96]:
notkeepprovince_measures_df = pd.concat([country_notkeepprovince_measures_df, filtered_province_notkeepprovince_measures_df])
notkeepprovince_measures_df

,COUNTRY,REGION,LOG_TYPE,CATEGORY,MEASURE,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
11,Afghanistan,Asia,Introduction / extension of measures,Public health measures,awareness campaigns,-,2020-01-24,2020-04-07 00:00:00
12,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-26,2020-04-07 00:00:00
13,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,international flights suspension,-,2020-01-27,2020-04-07 00:00:00
14,Afghanistan,Asia,Introduction / extension of measures,Public health measures,health screenings in airports and border cross...,-,2020-01-27,2020-04-07 00:00:00
10,Afghanistan,Asia,Introduction / extension of measures,Movement restrictions,border checks,-,2020-02-01,2020-03-14 00:00:00
...,...,...,...,...,...,...,...,...
492,Switzerland,Europe,Introduction / extension of measures,Social distancing,closure of businesses and public services,Jura,2020-04-24,2020-04-25 00:00:00
493,Switzerland,Europe,Phase-out measure,Social distancing,closure of businesses and public services,Wallis,2020-04-20,2020-04-25 00:00:00
494,Switzerland,Europe,Introduction / extension of measures,Governance and socio-economic measures,economic measures,Thurgau,2020-03-13,2020-04-25 00:00:00
554,Yemen,Middle East,Introduction / extension of measures,Social distancing,closure of businesses and public services,Internationally Recognized Government,2020-03-16,2020-04-09 00:00:00


In [97]:
grouped_notkeepprovince_measures_df = notkeepprovince_measures_df.groupby(['COUNTRY', 'MEASURE', 'LOG_TYPE']).first()
grouped_notkeepprovince_measures_df = grouped_notkeepprovince_measures_df.reset_index()
grouped_notkeepprovince_measures_df.shape

(3610, 8)

In [98]:
grouped_notkeepprovince_measures_df['ADMIN_LEVEL_NAME'].unique()

array(['-', 'Tyrol', 'Republika Srpska', 'FBiH', 'Srpska', 'Brcko',
       'Santiago and Boa Vista',
       'São Vicente, Santo Antão, São Nicolau, Sal, Maio, Fogo, and Brava',
       'Kinshasa', 'Lautoka', 'Greater Kumasi Metropolitan Area',
       'Greater Accra Metropolitan Area & Kasao', 'Najaf', 'Almaty',
       'Male', 'State of Mexico', ' Jalisco state', 'Bauchi', 'Medina',
       'Internationally Recognized Government + defacto'], dtype=object)

#### Don't keep province info in notkeep df

In [99]:
grouped_notkeepprovince_measures_df['ADMIN_LEVEL_NAME'] = "-"
grouped_notkeepprovince_measures_df

,COUNTRY,MEASURE,LOG_TYPE,REGION,CATEGORY,ADMIN_LEVEL_NAME,DATE_IMPLEMENTED,ENTRY_DATE
0,Afghanistan,awareness campaigns,Introduction / extension of measures,Asia,Public health measures,-,2020-01-24,2020-04-07 00:00:00
1,Afghanistan,border checks,Introduction / extension of measures,Asia,Movement restrictions,-,2020-02-01,2020-03-14 00:00:00
2,Afghanistan,border closure,Introduction / extension of measures,Asia,Movement restrictions,-,2020-02-24,2020-03-14 00:00:00
3,Afghanistan,border closure,Phase-out measure,Asia,Movement restrictions,-,2020-05-14,2020-05-18 00:00:00
4,Afghanistan,changes in prison-related policies,Introduction / extension of measures,Asia,Social distancing,-,2020-03-31,2020-05-11 00:00:00
...,...,...,...,...,...,...,...,...
3605,Zimbabwe,other public health measures enforced,Phase-out measure,Africa,Public health measures,-,2020-04-03,2020-04-07 00:00:00
3606,Zimbabwe,requirement to wear protective gear in public,Introduction / extension of measures,Africa,Public health measures,-,2020-05-04,2020-05-25 00:00:00
3607,Zimbabwe,schools closure,Introduction / extension of measures,Africa,Social distancing,-,2020-03-24,2020-03-24 00:00:00
3608,Zimbabwe,state of emergency declared,Introduction / extension of measures,Africa,Governance and socio-economic measures,-,2020-03-17,2020-04-10 00:00:00


### Preprocessing - keep province group

In [100]:
grouped_keepprovince_measures_df = keepprovince_measures_df.groupby(['COUNTRY', 'MEASURE', 'LOG_TYPE', 'ADMIN_LEVEL_NAME']).first()
grouped_keepprovince_measures_df = grouped_keepprovince_measures_df.reset_index()
print('Shape: {}'.format(grouped_keepprovince_measures_df.shape))
grouped_keepprovince_measures_df

Shape: (318, 8)


,COUNTRY,MEASURE,LOG_TYPE,ADMIN_LEVEL_NAME,REGION,CATEGORY,DATE_IMPLEMENTED,ENTRY_DATE
0,Australia,awareness campaigns,Introduction / extension of measures,-,Pacific,Public health measures,2020-03-17,2020-03-21 00:00:00
1,Australia,awareness campaigns,Introduction / extension of measures,NSW,Pacific,Public health measures,2020-04-08,2020-04-09 00:00:00
2,Australia,awareness campaigns,Introduction / extension of measures,NT,Pacific,Public health measures,2020-03-26,2020-03-27 00:00:00
3,Australia,awareness campaigns,Introduction / extension of measures,WA,Pacific,Public health measures,2020-04-27,2020-04-27 00:00:00
4,Australia,border closure,Introduction / extension of measures,-,Pacific,Movement restrictions,2020-03-20,2020-03-21 00:00:00
...,...,...,...,...,...,...,...,...
313,United Kingdom,schools closure,Introduction / extension of measures,-,Europe,Social distancing,2020-03-20,2020-03-23 00:00:00
314,United Kingdom,schools closure,Phase-out measure,England,Europe,Social distancing,2020-06-01,2020-06-02 00:00:00
315,United Kingdom,strengthening the public health system,Introduction / extension of measures,-,Europe,Public health measures,2020-03-11,2020-03-15 00:00:00
316,United Kingdom,surveillance and monitoring,Introduction / extension of measures,-,Europe,Movement restrictions,2020-03-24,2020-03-24 00:00:00


### Concat both notkeep province & keep province together

In [101]:
measure_df = pd.concat([grouped_keepprovince_measures_df, grouped_notkeepprovince_measures_df])
measure_df = measure_df.reset_index().drop(['index'], axis=1)
measure_df

,COUNTRY,MEASURE,LOG_TYPE,ADMIN_LEVEL_NAME,REGION,CATEGORY,DATE_IMPLEMENTED,ENTRY_DATE
0,Australia,awareness campaigns,Introduction / extension of measures,-,Pacific,Public health measures,2020-03-17,2020-03-21 00:00:00
1,Australia,awareness campaigns,Introduction / extension of measures,NSW,Pacific,Public health measures,2020-04-08,2020-04-09 00:00:00
2,Australia,awareness campaigns,Introduction / extension of measures,NT,Pacific,Public health measures,2020-03-26,2020-03-27 00:00:00
3,Australia,awareness campaigns,Introduction / extension of measures,WA,Pacific,Public health measures,2020-04-27,2020-04-27 00:00:00
4,Australia,border closure,Introduction / extension of measures,-,Pacific,Movement restrictions,2020-03-20,2020-03-21 00:00:00
...,...,...,...,...,...,...,...,...
3923,Zimbabwe,other public health measures enforced,Phase-out measure,-,Africa,Public health measures,2020-04-03,2020-04-07 00:00:00
3924,Zimbabwe,requirement to wear protective gear in public,Introduction / extension of measures,-,Africa,Public health measures,2020-05-04,2020-05-25 00:00:00
3925,Zimbabwe,schools closure,Introduction / extension of measures,-,Africa,Social distancing,2020-03-24,2020-03-24 00:00:00
3926,Zimbabwe,state of emergency declared,Introduction / extension of measures,-,Africa,Governance and socio-economic measures,2020-03-17,2020-04-10 00:00:00


In [102]:
lockdown_measure_df = measure_df[measure_df['MEASURE'] == 'lockdown']
lockdown_measure_df.shape

(236, 8)

In [103]:
measure_df.head()

,COUNTRY,MEASURE,LOG_TYPE,ADMIN_LEVEL_NAME,REGION,CATEGORY,DATE_IMPLEMENTED,ENTRY_DATE
0,Australia,awareness campaigns,Introduction / extension of measures,-,Pacific,Public health measures,2020-03-17,2020-03-21 00:00:00
1,Australia,awareness campaigns,Introduction / extension of measures,NSW,Pacific,Public health measures,2020-04-08,2020-04-09 00:00:00
2,Australia,awareness campaigns,Introduction / extension of measures,NT,Pacific,Public health measures,2020-03-26,2020-03-27 00:00:00
3,Australia,awareness campaigns,Introduction / extension of measures,WA,Pacific,Public health measures,2020-04-27,2020-04-27 00:00:00
4,Australia,border closure,Introduction / extension of measures,-,Pacific,Movement restrictions,2020-03-20,2020-03-21 00:00:00


#### Change column name

In [105]:
measure_df.rename(columns={'LOG_TYPE':'TYPE','ADMIN_LEVEL_NAME':'LEVEL'}, inplace=True)

In [107]:
measure_df.to_csv('dataset/measures.csv')

In [ ]:
measure_df['COUNTRY'].unique()

In [44]:
lockdown_measure_df.head()

,COUNTRY,MEASURE,LOG_TYPE,ADMIN_LEVEL_NAME,REGION,TYPE,CATEGORY,LEVEL,DATE_IMPLEMENTED,ENTRY_DATE
11,Australia,lockdown,*,-,Pacific,Introduction / extension of measures,Lockdown,*,2020-03-23,2020-04-02 00:00:00
57,China,lockdown,*,-,Asia,Phase-out measure,Lockdown,Suifenhe,2020-05-07,2020-05-11 00:00:00
58,China,lockdown,Introduction / extension of measures,-,*,*,Lockdown,*,2020-01-23,1/23/2020
59,China,lockdown,Introduction / extension of measures,Hubei,Hubei,*,Lockdown,*,2020-01-23,1/23/2020
90,France,lockdown,*,-,Europe,Introduction / extension of measures,Lockdown,*,2020-03-17,2020-03-17 00:00:00


### Insert "confirmed" dataset

In [78]:
table_name = 'confirmed'
# use "multi" method for insert many value in one statement sql
confirmed_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

In [90]:
table_name = 'measures'
# use "multi" method for insert many value in one statement sql
measure_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

In [91]:
table_name = 'lockdown_measures'
# use "multi" method for insert many value in one statement sql
lockdown_measure_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

In [9]:
table_name = 'us_state_name'
# use "multi" method for insert many value in one statement sql
us_state_name_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

In [75]:
table_name = 'confirmed_no_recov'
# use "multi" method for insert many value in one statement sql
covid_no_recov_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")

In [74]:
table_name = 'confirmed'
# use "multi" method for insert many value in one statement sql
confirmed_df.to_sql(table_name, con, schema=schema, if_exists="replace", method="multi")